In [1]:
import pandas as pd
from app import app, db, Reservation, DeviceStatus, User
from datetime import datetime, timedelta
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [2]:
xls = pd.ExcelFile('data/dataset.xlsx')
df1 = pd.read_excel(xls, 'Formulierreacties 1')

In [3]:
df1 = df1.drop(['Time of visit', 'Date of visit ', 'Free space for comments', 'Which game are you going to play?', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12','Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'], axis=1)

In [11]:
SELECT name FROM sqlite_master WHERE type='table';

SyntaxError: invalid syntax (2068583265.py, line 1)

In [4]:
df1

,Tijdstempel,Full name,Student number,What PC/Console will you use?,"Which university are you from? (If not from a uni, please let us know from where at the ""other"" section)",E-mailadres
0,2022-09-19 15:41:23.238,Jian Jin He,193099,NaN,NaN,NaT
1,2022-09-19 15:41:27.396,Thomas Pichardo,223834,NaN,NaN,NaT
2,2022-09-19 15:41:31.151,Samuel Antonio Vieira Vasconcelos,211941,NaN,NaN,NaT
3,2022-09-19 15:41:54.527,Juan José Gutierrez,215517,NaN,NaN,NaT
4,2022-09-19 15:42:06.884,Fedor Chursin,220905,NaN,NaN,NaT
...,...,...,...,...,...,...
2178,2024-09-02 11:37:35.030,Onur,223335,04,Buas,NaT
2179,2024-09-02 11:38:08.775,Yord van Breugel,222640,03,Buas,NaT
2180,2024-09-02 11:40:05.974,Maurits Peij,234182,07,Buas,NaT
2181,2024-09-02 11:40:32.939,Achilleas Lordos,230746,09,Buas,NaT


In [5]:
# Rename columns to match SQLAlchemy model
df1.rename(columns={
    'Full name': 'name',
    'Which university are you from? (If not from a uni, please let us know from where at the "other" section)': 'school',
    'E-mailadres': 'email',
    'What PC/Console will you use?': 'device',
    'Tijdstempel': 'date'
}, inplace=True)

In [6]:
df1

,date,name,Student number,device,school,email
0,2022-09-19 15:41:23.238,Jian Jin He,193099,NaN,NaN,NaT
1,2022-09-19 15:41:27.396,Thomas Pichardo,223834,NaN,NaN,NaT
2,2022-09-19 15:41:31.151,Samuel Antonio Vieira Vasconcelos,211941,NaN,NaN,NaT
3,2022-09-19 15:41:54.527,Juan José Gutierrez,215517,NaN,NaN,NaT
4,2022-09-19 15:42:06.884,Fedor Chursin,220905,NaN,NaN,NaT
...,...,...,...,...,...,...
2178,2024-09-02 11:37:35.030,Onur,223335,04,Buas,NaT
2179,2024-09-02 11:38:08.775,Yord van Breugel,222640,03,Buas,NaT
2180,2024-09-02 11:40:05.974,Maurits Peij,234182,07,Buas,NaT
2181,2024-09-02 11:40:32.939,Achilleas Lordos,230746,09,Buas,NaT


In [7]:
# Create SQLAlchemy engine
engine = create_engine('sqlite:///reservations.db')
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
def map_df_to_model(df):
    reservations = []
    for index, row in df.iterrows():
        # Create a Reservation instance
        reservation = Reservation(
            name=row['name'],
            school=row['school'],
            email=row['email'],
            device=row['device'],
            pin_hash='',  # You may need to generate or calculate a PIN hash here
            date=row['date'].to_pydatetime()  # Convert Timestamp to datetime object
        )
        reservations.append(reservation)
    return reservations

In [9]:
# Map DataFrame to model instances
reservations = map_df_to_model(df1)

# Add instances to the session and commit
try:
    session.add_all(reservations)
    session.commit()
    print("Data inserted successfully.")
except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    session.close()

An error occurred: (sqlite3.OperationalError) no such table: reservation
[SQL: INSERT INTO reservation (name, school, email, device, pin_hash, date) VALUES (?, ?, ?, ?, ?, ?) RETURNING id]
[parameters: ('Jian Jin He', nan, NaT, nan, '', '2022-09-19 15:41:23.238000')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
# Check current database entries
with app.app_context():
    # Query all reservations
    reservations = Reservation.query.all()
    print("Reservations:", reservations)

    # Query all device statuses
    device_statuses = DeviceStatus.query.all()
    print("Device Statuses:", device_statuses)

    # Query all users
    users = User.query.all()
    print("Users:", users)

    # Query reservations in the last 24 hours
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)
    reservations_last_24_hours = Reservation.query.filter(
        Reservation.date.between(start_date, end_date)
    ).all()
    print("Reservations in Last 24 Hours:", reservations_last_24_hours)

Reservations: []
Device Statuses: []
Users: []
Reservations in Last 24 Hours: []
